In [1]:
import pandas
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time
from pandas import DataFrame
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

records = []
links_reportagem = []

session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

driver = webdriver.Chrome('C:/chromedriver/chromedriver.exe')

def limpaArrays(array):
    igual = set()
    result = []
    for item in array:
        if item not in igual:
            igual.add(item)
            result.append(item)
    return result

def gera_json():
    records_limpos = limpaArrays(records)

    df = DataFrame(records_limpos, columns=['comentarios'])
    with open('pacote_mais_lidas_g1.json', 'w', encoding='utf-8') as file:
        df.to_json(file, force_ascii=False, orient='index')

def pega_comentarios():
    res = driver.execute_script('return document.documentElement.outerHTML')
    soup = BeautifulSoup(res, 'html.parser') 
    lista_recente = soup.find("div","glbComentarios-lista glbComentarios-lista-recentes")
    lista = lista_recente.find("ul", class_ = "glbComentarios-lista-todos")
    if(lista):
        li = lista.find_all("li", {"itemtype":"http://schema.org/UserComments"})
        for coments_ in li:
            coments = coments_.find("p", class_ = "glbComentarios-texto-comentario")
            if(coments):
                comentarios = coments.contents[0]
                comentarios.strip()
                records.append((comentarios)) 

def botao_respostas():
    while True:
        try:
            mais_comentarios = driver.find_element_by_xpath("//*[@class='glbComentarios-lista glbComentarios-lista-recentes']/ul/li/div[1]/div/div[3]/button[not(contains(@style,'display: none'))]")
            time.sleep(3)
            if(mais_comentarios.is_displayed()):
                ActionChains(driver).move_to_element(mais_comentarios).click(mais_comentarios).perform()
        except NoSuchElementException as e:
            break
    pega_comentarios()   

def botao_carrega():
    while True:
        try:
            carrega_mais = driver.find_element_by_xpath('//*[@id="boxComentarios"]/div[4]/button')
            time.sleep(2)
            if(carrega_mais.is_displayed()):
                ActionChains(driver).move_to_element(carrega_mais).click(carrega_mais).perform()
            else:
                break
        except NoSuchElementException as e:
            break
    botao_respostas()

def busca_reportagens(url):
    driver.get(url)
    driver.maximize_window()
    driver.refresh()
    time.sleep(3)
    res = driver.execute_script('return document.documentElement.outerHTML')
    soup = BeautifulSoup(res, 'html.parser') 
    div_maisLidas = soup.find('div', 'post-bastian-products post-mais-lidas theme')
    section_maisLidas = div_maisLidas.find_all('section',{'class':'post-bastian-products__section post-mais-lidas__section'})
    for mais_lidas in section_maisLidas:
        tag_a = mais_lidas.find("a")
        links_reportagem.append(tag_a.get('href'))
        
    for links in limpaArrays(links_reportagem):
        try:
            pagina = session.get(links)
        except:
            time.sleep(5)
        finally:
            soup = BeautifulSoup(pagina.text, 'html.parser')
            existe_boxcomentarios = soup.find("div",{"id":"boxComentarios"})
            if(existe_boxcomentarios):
                driver.get(links)
                driver.maximize_window()
                try:
                    box_comentarios = driver.find_element_by_id('boxComentarios')
                    driver.execute_script('arguments[0].scrollIntoView(true);', box_comentarios)
                finally:
                    try:
                        element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.CLASS_NAME, "glbComentarios"))
                        )
                    finally:
                        botao_carrega()
    driver.quit()


def __init__(url):
    busca_reportagens(url)
    
__init__('https://g1.globo.com')
gera_json()   
        
